# "Предобработка"

In [252]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [253]:
vals = '0.78 3 0.4 9 0.61 7 4.5 20 0.41 2.8 4 10 0.08 3.4 3.8 10 0.03 1.3 2.2 6 0.52 8.3 6.2 100 0.19 2 1.8 1 0.78 4.5 2.7 18 0.05 2.1 5 20 0.31 8 6.8 35 0.65 4 4.2 17 0.39 7.5 5.5 30 0.64 5 5.2 21 0.97 2.3 1.4 2 0.59 7.1 5.1 25 0.97 3.4 0.2 5 0.97 9 5.7 70 0.20 0.4 7 13 0.48 6 1 16 0.47 1 3.3 1'

так как нет никаких предположений о данных (это инопланентная цивилизация, может быть что угодно), то нам совершенно ничего не дает знание о том, что первая фича находится в отрезке [0, 1], поэтому для удобства скалируем ее до range остальных фичей.

In [254]:
L = list(map(float, vals.split(' ')))
df = pd.DataFrame([L[i:i+4] for i in range(0, len(L), 4)], columns=['f1','f2','f3','price'])
df.f1*=10

# Попытка анализировать признаки:

Нет смысла разбивать на Train, Test так как данных почти нет.

## PCA

In [247]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
pca.fit(df.to_numpy())
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

[0.97524651 0.01670563 0.00505649 0.00299137]
[105.74640567  13.84011856   7.61435458   5.85656984]


=> не можем утверждать, что некоторые фичи шум (конечно, всего же 20 строчек дано...)

## Lasso

In [250]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(df.iloc[:,:-1], df.iloc[:,-1])
print(clf.coef_)

[0.61106979 5.15951444 4.06085122]


Lasso не зануляет ни одну из фичей, поэтому опять же не можем утверждать бесполезность первой фичи.

## Надежда на линейность

In [251]:
from sklearn.metrics import r2_score
y_pred = linear_model.LinearRegression().fit(df.iloc[:,:-1], df.iloc[:,-1]).predict(df.iloc[:,:-1])
r2_score(df.iloc[:,-1], y_pred)

0.6400517421474177

Безнадежно, r2_score сильно недостаточный для статистической значимости того, что данную функцию можно подобрать гиперплоскостью

# Предположение: цена камня вычисляется как линейная функция от f(фичи), где f(x) - базовые функции вида
# $x,\ x^2,\ x^3,\ \sqrt{x},\ \sqrt[3]{x},\ sin(x),\ cos(x)$
(то есть понадеемся, что инопланетяне - обычные люди, использующие такие простые функции для рассчета стоимости камня)

In [255]:
for i in range(1,4):
  df[f'f{i}^2'] = df[f'f{i}']**2
  df[f'f{i}^0.5'] = df[f'f{i}']**(0.5)
  df[f'sin(f{i})'] = np.sin(df[f'f{i}'])
  df[f'cos(f{i})'] = np.cos(df[f'f{i}'])
  df[f'f{i}^3'] = df[f'f{i}']**3 
  df[f'f{i}^0.33'] = df[f'f{i}']**(1/3)
df = df[[i for i in df.columns if i!='price']+['price']]

In [262]:
from sklearn.model_selection import train_test_split
traincols = [i for i in df.columns if i not in ['price']]
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, traincols], df['price'], test_size=0.33, random_state=42)
X_full, y_full = df.loc[:, traincols], df.loc[:,'price']

Сначала посмотрим на регрессию на всех данных и подумаем, будто мы нашли эту самую f:

In [260]:
linreg = linear_model.LinearRegression().fit(X_full, y_full)
y_pred = linreg.predict(X_full)
r2_score(y_full, y_pred)

1.0

мы в точности подобрали функцию для аппроксимации!!! - нет, это лишь оверфит в 20 строк данных... (посмотрим на трейн-тест вариант)

In [263]:
linreg = linear_model.LinearRegression().fit(X_train, y_train)
y_pred = linreg.predict(X_test)
r2_score(y_test, y_pred)

0.5567816508122327

# Предположение 2: в ценообразование f(x) входят нелинейности вида "если фича 1>0.5, то f(x), иначе g(x)"


К сожалению, 20 строк никак не хватит чтобы статистически проверить данное предположение.

# Что остается?

Мой алгоритм для Элли:\
пока пользоваться простой "линейной" функцией из предположения 1, и при добавлении новых данных проверять гипотезу что это все еще та функция, а также попробовать использовать решающие деревья (при достаточно большом массиве данных)